In [ ]:
from __future__ import division
from datetime import datetime, timedelta
from time import sleep
import senticAPIextractor
import pandas as pd
import FilSentimentExtract 
#import matplotlib.pyplot as plt
#from stockstats import StockDataFrame 
import numpy as np
#import pysentiment as ps
#from collections import OrderedDict
import re
import senticAPIextractor
from os import listdir
import random


INITIAL_WEIGHT = 1;



indexesFiles=list()
newsFiles=list()
files=list()
newsFiles=listdir("/home/andrea/Desktop/NLFF/intrinioDataset")
indexesFiles=listdir("/home/andrea/Desktop/NLFF/DataSetIndexes")
#print(newsFiles)
#print(indexesFiles)
tickers=list()
for file in indexesFiles:
    file=re.sub('\.csv$', '', file)
    file=re.sub('indexes', '', file)
    tickers.append(file)
tickers.sort()
print(tickers)

In [ ]:
#tickers=["AAPL"]

for ticker in tickers:
    data = list()
    timeSpan = list()
    print("Working on "+ticker+"...")
    
    #VARIABILE PER SETTARE DELAY DELLE NEWS IN MIN
    delay=20
    reader=pd.read_csv('/home/andrea/Desktop/NLFF/intrinioDataset/preprocessing/preprocessed/'+ticker+'.csv') #reader = csv.DictReader(csvfile)
    for row in reader.T.iteritems():
        time  = datetime.strptime(row[1]['PUBLICATION_DATE'], '%Y-%m-%d %H:%M:%S +%f')
        time=time+timedelta(minutes=delay)
        data.append({'time':time, 'summary':row[1]['SUMMARY']}) 


    reader = pd.read_csv('/home/andrea/Desktop/NLFF/DataSetIndexes/indexes'+ticker+'.csv')
    reader.rename(columns={'Unnamed: 0':'date'}, inplace=True)
    for row in reader.T.iteritems():
        time  = datetime.strptime(row[1]['date'], '%Y-%m-%d %H:%M:%S')
        timeSpan.append(time) 



    i = len(data)-1;
    j=0;
    
    #Nuova colonna al dataframe per il sentiment
    reader["sentiment"] = np.nan
    
    initDate = max(timeSpan[0], data[len(data)-1]['time'])
    finalDate = min(timeSpan[len(timeSpan)-1], data[0]['time'])
    
    # ALLINEAMENTO INIZIO
    while(timeSpan[j] < initDate):
        j+=1
    while(data[i]['time'] <= initDate):
        i-=1
        

    weighted_sum = 0;
    normal_sum = 0
    while( data[i]['time'] < finalDate and timeSpan[j] < finalDate ):
        
        total_subj = INITIAL_WEIGHT
        num_sentiment = 1
        
        initTime = timeSpan[j]
        
        while(i>0 and timeSpan[j] > data[i]['time']):
            if not (timeSpan[j] > data[i]['time'] and timeSpan[j-1] <= data[i]['time']):
                print("timeSpan["+str(j)+"]: "+str(timeSpan[j])+" data["+str(i)+"] : " +str(data[i]['time']) + " timeSpan["+str(j-1)+"]: "+str(timeSpan[j-1]))
                assert False
            try:
                subjectivity = 1
                
                polarity=FilSentimentExtract.estractSentimentFil(data[i]['summary'].decode('utf-8'))
                
                if(polarity==100 ):
                    
                    content=senticAPIextractor.extractPolarity(data[i]['summary'].decode('utf-8'))
                    if(content['polarity']=='positive'):
                        polarity=1
                    else:
                        polarity=-1

                normal_sum += polarity

                weighted_sum += polarity*subjectivity
                
                total_subj+=subjectivity
                num_sentiment +=1
            except Exception as e: 
                    print(e)
                    #print(data[i]['summary'])

            i-=1
            
        weighted_sum /=total_subj
        normal_sum /=num_sentiment
        reader["sentiment"].at[j] = normal_sum    
        j+=1

        print('normal sum: '+str(normal_sum))

    #Write results
    reader = reader[pd.isnull(reader['sentiment']) == False ]
    
    reader.to_csv('/home/andrea/Desktop/NLFF/DataSetSentimentWithFil/sentiment20minutes'+ticker+'.csv')
    